## Blue Stragglers Research Project
Yuvraj Sahu, Elijah Flores, Andrew Harvey

Geometry of Space - Freshman Research Initiative

The University of Texas at Austin

In [ ]:
# Import libraries
import pandas as pd
import plotly.graph_objects as go
import requests
import concurrent.futures
import requests

In [ ]:
# Global parameters, can be changed for customization or use of different clusters
file_name       = 'NGC 2682.csv'
cluster_name    = 'NGC 2682'
df              = pd.read_csv(file_name, skiprows=0)
bss_color       = 'blue'
candidate_color = 'skyblue'
default_color   = 'black'
error_color     = 'lightgrey'

In [ ]:
# Creates Simbad URL from a GAIA ID
def link(id):
    u = id.split()
    return 'http://simbad.cds.unistra.fr/simbad/sim-basic?Ident=' + u[0] + '+' + u[1] + '+' + u[2] + '&submit=SIMBAD+search'

In [ ]:

# Fetches color of object
def fetch_star_color(star):
    url = link(df['designation'][star])
    try:
        response = requests.get(url, timeout=2)
        html_string = response.text
        i = html_string.find(' is a ')
        j = html_string.find('"', i)
        star_type = html_string[i+6:j]
        if star_type == 'Blue Straggler Candidate':
            return candidate_color
        elif star_type == 'Blue Straggler':
            return bss_color
        else:
            return default_color
    except requests.exceptions.Timeout:
        print(f'Could not get request for {df["designation"][star]}')
        return error_color

# Determines BSS candidacy for each star
def determine_candidacy():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        colors = list(executor.map(fetch_star_color, range(len(df.index))))
    df['color'] = colors


In [ ]:
# Creates graph objects
fig = go.FigureWidget([go.Scatter(x=df['g_rp'], y=df['phot_g_mean_mag'], mode='markers')])
scatter = fig.data[0]

# Updates graph object properties
determine_candidacy()
scatter.marker.color = df['color']
scatter.marker.size = [10] * len(df.index)
fig.update_layout(dict(yaxis_autorange = 'reversed', xaxis_title = 'G - Rp [mag]', yaxis_title = 'G [mag]', title_text = 'Color-Magnitude Diagram of ' + cluster_name, title_x = 0.5))
fig.layout.hovermode = 'closest'

# Displays graph
fig

In [ ]:
# Updates the clicked on point, resets any previous points, and prints out
# star information
def update_point(trace, points, selector):
    marker_sizes = [10] * len(df.index)
    clicked_pt = points.point_inds[0]
    marker_sizes[clicked_pt] = 15
    scatter.marker.size = marker_sizes
    print(str(df['designation'][clicked_pt]))
    print('RA: ' + str(df['ra'][clicked_pt]))
    print('Dec: ' + str(df['dec'][clicked_pt]))
    print('Simbad URL: ' + link(df['designation'][clicked_pt]))
    print()
scatter.on_click(update_point)